In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_vertical_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,24007.28,-0.08,0.01,-0.05,-0.03,-0.00,-0.00,0.00
3,-0.08,5740.93,-0.01,0.03,0.00,0.00,0.00,-0.00
4,0.01,-0.01,1135.03,-0.02,-0.00,0.00,-0.00,0.00
5,-0.05,0.03,-0.02,213.42,0.00,0.00,0.00,-0.00
6,-0.03,0.00,-0.00,0.00,36.27,-0.00,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,5.65,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.88,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00002,-0.00003,-0.00001,-0.00002,0.00000
3,-0.00001,1.00000,-0.00000,0.00002,0.00001,0.00002,0.00000,-0.00001
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00004,-0.00001,0.00001
5,-0.00002,0.00002,-0.00004,1.00000,0.00002,0.00000,0.00001,-0.00002
6,-0.00003,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00002,0.00003
7,-0.00001,0.00002,0.00004,0.00000,-0.00001,1.00000,-0.00001,-0.00001
8,-0.00002,0.00000,-0.00001,0.00001,0.00002,-0.00001,1.00000,0.00001
9,0.00000,-0.00001,0.00001,-0.00002,0.00003,-0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.1462925120613544

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.21362439e+07 4.03550101e+06 1.53419309e+05 5.20744339e+03
 1.41082226e+02 3.05042817e+00 5.58069828e-02 6.35867167e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999323,-0.036153,-0.006697,-0.001258,-0.000215,-0.000033,-0.000005,-7.074323e-07
1,0.035892,0.998740,-0.034496,-0.006195,-0.001051,-0.000163,-0.000025,-3.585177e-06
2,0.007870,0.033958,0.998604,-0.039158,-0.006381,-0.000989,-0.000155,-2.128448e-05
3,0.001774,0.007414,0.038590,0.998239,-0.043905,-0.006598,-0.001024,-1.426829e-04
4,0.000377,0.001570,0.007957,0.043195,0.997577,-0.053327,-0.008023,-1.111098e-03
5,0.000078,0.000324,0.001637,0.008725,0.052168,0.995379,-0.079421,-1.063020e-02
6,0.000018,0.000074,0.000377,0.002003,0.011794,0.076659,0.987293,-1.386785e-01
7,0.000005,0.000021,0.000106,0.000566,0.003324,0.021359,0.137399,9.902798e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006769727534459147,
 0.001259656693743949,
 0.001396053139721798,
 0.0017613234611053574,
 0.0024234997806765746,
 0.004621254186551793,
 0.0127065119517914,
 0.009720235946462763]